In [11]:
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display
from jax.tree_util import tree_map 

In [12]:
HOME = os.environ.get('HOME')


import analysis as A
import common as C

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
mimic_dir = {
    'M3': f'{HOME}/GP/ehr-data/mimic3-transforms',
    'M4': f'{HOME}/GP/ehr-data/mimic4-transforms'
}

train_dir = {
    'GM3': f'{HOME}/GP/ehr-data/icenode-m3-exp/train_config_v0.2.25_G_M3',
    'M3': f'{HOME}/GP/ehr-data/icenode-m3-exp/train_config_v0.2.25_M3',
    'M4': f'{HOME}/GP/ehr-data/icenode-m4-exp/train_config_v0.2.25_M4',
    'GM4': f'{HOME}/GP/ehr-data/icenode-m4-exp/train_config_v0.2.25_G_M4'
}

model_dir = {
    'ICE-NODE': 'icenode_2lr',
    'ICE-NODE_UNIFORM': 'icenode_uniform2lr',
    'GRU': 'gru',
    'RETAIN': 'retain'
}


clfs = list(model_dir.keys())


In [30]:

def fix(clfs, train_dir, model_dir, data_tag):
    clfs_params_dir = train_dir[data_tag]
    for clf in clfs:
        clf_dir = model_dir[clf]
        clf_cls = C.model_cls[clf]
        param_files = sorted(
            glob.glob(f'{clfs_params_dir}/{clf_dir}/*params.pickle', recursive=False))
        
        for param_file in param_files:
            params = C.load_params(param_file)
            if 'diag_emb' in params:
                print('x')
                params['dx_emb'] = params['diag_emb']
                del params['diag_emb']
                C.write_params(params, param_file)
        
        if len(param_files) > 0:
            config = C.load_config(f'{clfs_params_dir}/{clf_dir}/config.json')
        
            if 'emb' in config:
                if 'diag' in config['emb']:
                    config['emb']['dx'] = config['emb']['diag']
                    del config['emb']['diag']

            C.write_config(config, f'{clfs_params_dir}/{clf_dir}/config.json')
        
        
        

In [31]:
fix(clfs, train_dir, model_dir, 'M3')

In [32]:
fix(clfs, train_dir, model_dir, 'M4')

In [33]:
fix(clfs, train_dir, model_dir, 'GM3')

In [34]:
fix(clfs, train_dir, model_dir, 'GM4')